## 1️⃣ Instalar dependencias (5-10 minutos)

Ejecuta esta celda siempre (primera vez y actualizaciones)

In [ ]:
# Instalar herramientas necesarias
!apt-get update -qq
!apt-get install -y -qq git zip unzip openjdk-17-jdk autoconf libtool pkg-config \
    zlib1g-dev libncurses5-dev libncursesw5-dev libtinfo5 cmake libffi-dev libssl-dev

# Instalar Buildozer y Cython
!pip install -q buildozer cython==0.29.33

print("\n✅ Dependencias instaladas correctamente")
print("📦 Versiones:")
!buildozer version
!python3 --version

## 2️⃣ Subir proyecto (OPCIÓN A: ZIP completo)

**Recomendado para primera vez o cambios mayores**

Ejecuta el script `crear_zip_para_colab.ps1` en tu PC para crear el ZIP.

In [ ]:
from google.colab import files
import zipfile
import os
import shutil

# Crear directorio del proyecto
!rm -rf /content/app
!mkdir -p /content/app
%cd /content/app

print("📦 Sube el archivo ZIP con todo el proyecto:")
print("   (Ejecuta crear_zip_para_colab.ps1 para generarlo)")
uploaded = files.upload()

if uploaded:
    # Obtener el nombre del archivo subido
    zip_filename = list(uploaded.keys())[0]
    
    # Descomprimir
    print(f"\n📂 Descomprimiendo {zip_filename}...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('.')
    
    # Eliminar el ZIP
    os.remove(zip_filename)
    
    # CORREGIR: Reorganizar archivos si tienen rutas de Windows
    print("\n🔧 Reorganizando archivos...")
    for item in os.listdir('.'):
        # Si hay archivos con backslash en el nombre (ej: 'models\__init__.py')
        if '\\' in item:
            # Extraer la ruta correcta
            parts = item.split('\\')
            if len(parts) > 1:
                # Crear directorio si no existe
                dir_path = parts[0]
                os.makedirs(dir_path, exist_ok=True)
                # Mover archivo a su ubicación correcta
                new_path = os.path.join(*parts)
                if os.path.isfile(item):
                    shutil.move(item, new_path)
                    print(f"  Movido: {item} -> {new_path}")
    
    print("\n✅ Proyecto descomprimido y reorganizado correctamente")
    print("\n📂 Estructura del proyecto:")
    !ls -lhR
    
    # Verificar archivos principales
    print("\n📋 Verificación de archivos:")
    archivos_requeridos = ['main.py', 'buildozer.spec', 'requirements.txt']
    for archivo in archivos_requeridos:
        if os.path.exists(archivo):
            print(f"✅ {archivo} encontrado")
        else:
            print(f"❌ {archivo} NO encontrado")
    
    # Verificar directorios
    directorios_requeridos = ['models', 'services', 'ui', 'imagenes']
    for directorio in directorios_requeridos:
        if os.path.exists(directorio) and os.path.isdir(directorio):
            archivos = len([f for f in os.listdir(directorio) if os.path.isfile(os.path.join(directorio, f))])
            print(f"✅ {directorio}/ encontrado ({archivos} archivos)")
        else:
            print(f"⚠️ {directorio}/ NO encontrado")
else:
    print("❌ No se subió ningún archivo")

## 2️⃣ OPCIÓN B: Subir solo archivos modificados

**Para actualizaciones rápidas** - Solo sube los archivos que cambiaron

In [ ]:
from google.colab import files
import os

print("🔄 ACTUALIZACIÓN RÁPIDA")
print("Sube solo los archivos que modificaste:\n")

# Verificar que ya existe el proyecto
if not os.path.exists('/content/app'):
    print("❌ ERROR: No existe proyecto base.")
    print("   Primero debes hacer un BUILD COMPLETO (opción A)")
else:
    %cd /content/app
    
    print("Archivos Python principales:")
    print("  - main.py")
    print("  - models/text_model.py")
    print("  - models/transcription.py")
    print("  - services/audio_service_android.py")
    print("  - ui/microphone_screen.py")
    print("\nSube los archivos modificados (puedes subir varios):")
    
    uploaded = files.upload()
    
    if uploaded:
        # Mover archivos a sus ubicaciones correctas
        for filename in uploaded.keys():
            if '/' in filename:
                # Crear directorio si no existe
                dir_path = os.path.dirname(filename)
                os.makedirs(dir_path, exist_ok=True)
                print(f"✅ {filename} actualizado")
            else:
                print(f"✅ {filename} actualizado")
        
        print("\n✅ Archivos actualizados correctamente")
        print("⏭️ Ahora ve a la celda 5 (Build Incremental)")
    else:
        print("❌ No se subieron archivos")

## 3️⃣ Configurar buildozer.spec (solo primera vez)

Verifica la configuración de buildozer

In [ ]:
import os

%cd /content/app

# Mostrar configuración actual
if os.path.exists('buildozer.spec'):
    print("📄 Configuración de buildozer.spec:\n")
    
    with open('buildozer.spec', 'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines, 1):
            if any(keyword in line for keyword in ['title', 'package', 'version', 'requirements', 'permissions', 'api']):
                if not line.strip().startswith('#'):
                    print(f"{i:3d}: {line.rstrip()}")
    
    print("\n✅ Configuración verificada")
    print("\n📋 Características principales:")
    print("   - Versión: 0.2")
    print("   - Permisos: RECORD_AUDIO, INTERNET, STORAGE")
    print("   - API Target: 33 (Android 13)")
    print("   - API Mínimo: 21 (Android 5.0)")
else:
    print("❌ No se encontró buildozer.spec")

## 4️⃣ BUILD COMPLETO - Primera compilación (40-50 minutos) ⏳

**⚠️ SOLO PARA LA PRIMERA VEZ**

Ejecuta esta celda solo si:
- Es tu primera compilación
- Cambiaste dependencias en buildozer.spec
- El build incremental falló

In [ ]:
import time

print("🔨 BUILD COMPLETO - Iniciando compilación del APK...")
print(f"⏰ Hora de inicio: {time.strftime('%H:%M:%S')}")
print("\n⚠️  IMPORTANTE:")
print("   - Este proceso tarda 40-50 minutos")
print("   - NO cierres esta pestaña")
print("   - Puedes minimizar pero mantén la sesión activa")
print("\n" + "="*60 + "\n")

%cd /content/app

# Limpiar builds anteriores
!rm -rf .buildozer/android/platform/build-*
!rm -rf bin/

# Compilar con Buildozer
!buildozer -v android debug

print("\n" + "="*60)
print(f"⏰ Hora de finalización: {time.strftime('%H:%M:%S')}")
print("\n✅ BUILD COMPLETO finalizado")
print("\n📝 El APK incluye:")
print("   ✅ Reconocimiento de voz con SpeechRecognizer nativo")
print("   ✅ Texto a voz con TTS de Android")
print("   ✅ Interfaz completa con todas las pantallas")
print("   ✅ Navegación con botones de volver")

## 5️⃣ BUILD INCREMENTAL - Actualización rápida (5-10 minutos) 🚀

**⚡ PARA ACTUALIZACIONES**

Ejecuta esta celda cuando:
- Ya hiciste un build completo anteriormente
- Solo modificaste archivos .py
- Quieres actualizar el APK rápidamente

In [ ]:
import time
import os

print("⚡ BUILD INCREMENTAL - Actualización rápida")
print(f"⏰ Hora de inicio: {time.strftime('%H:%M:%S')}")

%cd /content/app

# Verificar que existe build anterior
if not os.path.exists('.buildozer'):
    print("\n❌ ERROR: No existe build anterior")
    print("   Debes ejecutar primero el BUILD COMPLETO (celda 4)")
else:
    print("\n📦 Detectado build anterior")
    print("🔄 Actualizando solo los archivos modificados...\n")
    print("="*60 + "\n")
    
    # Build incremental - solo recompila lo necesario
    # No limpiamos .buildozer para aprovechar cache
    !buildozer android debug
    
    print("\n" + "="*60)
    print(f"⏰ Hora de finalización: {time.strftime('%H:%M:%S')}")
    print("\n✅ BUILD INCREMENTAL completado")
    print("\n🎉 Tu APK ha sido actualizado rápidamente!")

## 6️⃣ Verificar y descargar APK 📥

Descarga el APK generado

In [ ]:
import os
from google.colab import files

%cd /content/app

# Verificar si existe el directorio bin
if os.path.exists('bin'):
    apk_files = [f for f in os.listdir('bin') if f.endswith('.apk')]
    
    if apk_files:
        apk_path = f'bin/{apk_files[0]}'
        apk_size = os.path.getsize(apk_path) / (1024 * 1024)  # Tamaño en MB
        
        print(f"✅ APK encontrado: {apk_files[0]}")
        print(f"📦 Tamaño: {apk_size:.2f} MB")
        print(f"📍 Ubicación: {apk_path}")
        print(f"\n📋 Información del APK:")
        print(f"   - Nombre: App Para Sordos")
        print(f"   - Versión: 0.2")
        print(f"   - Package: org.sordos.appparasordos")
        print("\n⬇️ Descargando APK...")
        
        files.download(apk_path)
        print("\n🎉 ¡APK descargado correctamente!")
        print("\n📱 Para instalar en Android:")
        print("   1. Transfiere el APK a tu teléfono")
        print("   2. Habilita 'Instalar apps de origen desconocido'")
        print("   3. Abre el APK y acepta los permisos")
        print("   4. ¡Disfruta de la app!")
    else:
        print("❌ No se encontró ningún APK en el directorio bin")
        print("Contenido de bin:")
        !ls -lh bin/
else:
    print("❌ No se encontró el directorio bin")
    print("Verifica que la compilación haya terminado correctamente")

## 🔧 Solución de problemas

### Error en compilación

Si hay errores, prueba estas soluciones:

In [ ]:
# OPCIÓN 1: Limpiar y reintentar
print("🧹 Limpiando builds anteriores...")

%cd /content/app

!rm -rf .buildozer/android/platform/build-*
!rm -rf bin/

print("\n✅ Limpieza completa")
print("\n⚠️ Ahora ejecuta la celda 4 (BUILD COMPLETO) nuevamente")

In [ ]:
# OPCIÓN 2: Limpiar TODO (reset completo)
print("🧹 RESET COMPLETO - Eliminando todo...")

%cd /content/app

!rm -rf .buildozer
!rm -rf bin

print("\n✅ Reset completo realizado")
print("\n⚠️ Ahora ejecuta la celda 4 (BUILD COMPLETO) nuevamente")
print("   Esto tomará el tiempo completo de compilación (~45 min)")

## 📊 Ver logs y estructura

Para debugging y verificación

In [ ]:
import os

%cd /content/app

print("📁 Estructura del proyecto:\n")
!tree -L 3 -I '__pycache__|*.pyc'

print("\n" + "="*60)
print("\n📝 Archivos principales:\n")

archivos = [
    'main.py',
    'buildozer.spec',
    'models/text_model.py',
    'services/audio_service_android.py',
    'ui/microphone_screen.py'
]

for archivo in archivos:
    if os.path.exists(archivo):
        size = os.path.getsize(archivo)
        print(f"✅ {archivo:40s} ({size:6d} bytes)")
    else:
        print(f"❌ {archivo:40s} (NO ENCONTRADO)")

In [ ]:
# Ver últimas líneas del log de buildozer
print("📝 Últimas 100 líneas del log de compilación:\n")
print("="*60 + "\n")

import os

if os.path.exists('.buildozer'):
    # Buscar el archivo de log más reciente
    !find .buildozer -name "*.log" -type f -exec ls -t {} + | head -1 | xargs tail -100
else:
    print("❌ No hay logs disponibles. Ejecuta primero una compilación.")

## 📱 Guía de instalación en Android

### Pasos para instalar el APK:

1. **Transferir APK**
   - Descarga el APK desde Colab
   - Envíalo a tu teléfono (email, cable USB, Drive, etc.)

2. **Habilitar instalación**
   - Ve a Ajustes → Seguridad
   - Activa "Orígenes desconocidos" o "Instalar apps desconocidas"

3. **Instalar**
   - Abre el archivo APK descargado
   - Toca "Instalar"
   - Acepta los permisos cuando se soliciten

4. **Permisos requeridos**
   - 🎤 Micrófono (para reconocimiento de voz)
   - 🌐 Internet (para servicios de transcripción)
   - 💾 Almacenamiento (para guardar configuración)

5. **Primera ejecución**
   - Abre la app "App Para Sordos"
   - Acepta los permisos
   - ¡Listo para usar!

### Características de la app:

- **🎤 Micrófono**: Transcripción de voz a texto en tiempo real
- **✍️ Texto**: Escribe texto y conviértelo a voz
- **🎨 Interfaz**: Diseño limpio y fácil de usar
- **🔄 Navegación**: Botones de volver en todas las pantallas


## 🎯 Resumen de comandos

### Primera vez (Build completo):
```
1. Celda 1: Instalar dependencias (5-10 min)
2. Celda 2 Opción A: Subir ZIP completo (2-3 min)
3. Celda 3: Verificar configuración (1 min)
4. Celda 4: BUILD COMPLETO (40-50 min)
5. Celda 6: Descargar APK (1 min)

Total: ~50-65 minutos
```

### Actualización (Build incremental):
```
1. Celda 1: Instalar dependencias (5-10 min)
2. Celda 2 Opción B: Subir archivos modificados (1 min)
3. Celda 5: BUILD INCREMENTAL (5-10 min)
4. Celda 6: Descargar APK (1 min)

Total: ~12-22 minutos
```

### ⚡ Tips para builds rápidos:

- Mantén la sesión de Colab abierta
- Usa build incremental para cambios pequeños
- Solo usa build completo cuando cambies dependencias
- Guarda el .buildozer en Drive para reutilizar (avanzado)
